<a href="https://colab.research.google.com/github/zhangxs131/Tensorflow_learning/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#面向初学者的keras
使用model.compile和model.fit实现训练

In [1]:
import tensorflow as tf

mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,y_train/255.0 #归一化
x_train.shape

11501568/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [5]:
model=tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2765 - accuracy: 0.9200
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1279 - accuracy: 0.9610
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0925 - accuracy: 0.9722
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0721 - accuracy: 0.9780
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0584 - accuracy: 0.9818


#针对专业人员使用keras

主要就是不用Sequential而是实现Class 实现model

In [41]:
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape,x_test.shape)
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

#增加一个channel维度

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
print(x_train.shape,x_test.shape)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [42]:
class MyModel(Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.conv1=Conv2D(32,3,activation='relu')
    self.flatten=Flatten()
    self.d1=Dense(128,activation='relu')
    self.d2=Dense(10)


  def call(self,x):
    x=self.conv1(x)
    x=self.flatten(x)
    x=self.d1(x)
    return self.d2(x)

model_2=MyModel()

In [43]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer=tf.keras.optimizers.Adam()

In [44]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [45]:
#使用tf.GradientTape训练模型

def train_step(images,labels):
  with tf.GradientTape() as tape:
    predictions=model_2(images,training=True)
    loss=loss_object(labels,predictions)
  gradients=tape.gradient(loss,model_2.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model_2.trainable_variables))

  train_loss(loss)
  train_acc(labels,predictions)

def test_step(images,labels):
  predictions=model_2(images,training=False)
  t_loss=loss_object(labels,predictions)

  test_loss(t_loss)
  test_acc(labels,predictions)


In [46]:
#训练和验证
Epochs=5

for epoch in range(Epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for images,labels in train_ds:
    train_step(images,labels)

  for images,labels in test_ds:
    test_step(images,labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_acc.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_acc.result() * 100}'
  )

Epoch 1, Loss: 0.13128912448883057, Accuracy: 96.07833862304688, Test Loss: 0.07249544560909271, Test Accuracy: 97.58999633789062
Epoch 2, Loss: 0.041752591729164124, Accuracy: 98.68500518798828, Test Loss: 0.051359452307224274, Test Accuracy: 98.16999816894531
Epoch 3, Loss: 0.02403854951262474, Accuracy: 99.20999908447266, Test Loss: 0.05412775278091431, Test Accuracy: 98.38999938964844
Epoch 4, Loss: 0.013445992954075336, Accuracy: 99.56500244140625, Test Loss: 0.05556425452232361, Test Accuracy: 98.4000015258789
Epoch 5, Loss: 0.01054174080491066, Accuracy: 99.63666534423828, Test Loss: 0.05971650779247284, Test Accuracy: 98.31999969482422


#总结

上面keras训练比较快捷方便，但model还是class构建熟悉一些吧，然后模型上，上面的直接flatten，然后MLP进行分类，下面的简单用了个3*3的卷积，然后进行展平，MLP分类

